<a href="https://colab.research.google.com/github/victoreduardo/fl_experiment_with_airtlab/blob/main/Paper_non_FL_%2B_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mlflow
!pip install flwr

In [ ]:
from typing import Dict, Optional, Tuple

import flwr as fl
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image

import mlflow

# Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
MODELS_PATH = '/content/drive/MyDrive/FL/'
SAVE_MODELS_PATH = '/content/drive/MyDrive/FL/model'
DATASET_PATH = '/content/drive/MyDrive/FL/data/AIRTLab/'
DATASET_VALID_PATH = '/content/drive/MyDrive/FL/data/AIRTLab/test/'
TARGET_SIZE = (120,160)
DESIRED_ACCURACY = 0.995

VIOLENCIA_DIR = 'violent'
N_VIOLENCIA_DIR = 'non-violent'

In [ ]:
!databricks configure --host https://community.cloud.databricks.com/
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/vess@cesar.school/non-FL-AIRTLab")

Username: vess@cesar.school
Password: 
Repeat for confirmation: 


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2157939619876676', experiment_id='2157939619876676', lifecycle_stage='active', name='/Users/vess@cesar.school/non-FL-AIRTLab', tags={'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'vess@cesar.school',
 'mlflow.ownerId': '8444989453852335'}>

In [ ]:
mlflow.end_run()
mlflow.start_run(run_name='load_vgg16')

<ActiveRun: >

### FL Utils

In [ ]:
def test_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_VALID_PATH,
                                             batch_size=32,
                                             class_mode='categorical',
                                             target_size=TARGET_SIZE)

def train_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_PATH + 'train/',
                                             batch_size = 32,
                                             class_mode = 'categorical',
                                             target_size =TARGET_SIZE)


### Models

In [ ]:
metrics = [
  'accuracy'
]

In [ ]:
def load_inceptionv3():
    local_weights_file = MODELS_PATH + 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = InceptionV3(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('mixed10')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

def load_mobilenetv2():
    local_weights_file = MODELS_PATH + 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5'
    pre_trained_model = MobileNetV2(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('out_relu')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

def load_resnet152v2():
    local_weights_file = MODELS_PATH + 'resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = ResNet152V2(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('post_bn')
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_vgg16():
    local_weights_file = MODELS_PATH + 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = VGG16(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('block5_pool')
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

### Metrics

In [ ]:
def save_model(model):
    diretorio = '/content/sample_data/output/'
    model.save(diretorio)
    converter_tflite = tf.lite.TFLiteConverter.from_saved_model(diretorio)
    tflite_model = converter_tflite.convert()
    with open(diretorio + '/incepv3.tflite', 'wb') as f:
        f.write(tflite_model)
    mlflow.tensorflow.log_model(tflite_model)


def save_summary(model):
    from contextlib import redirect_stdout
    temp_name = 'modelsummary.txt'
    with open(temp_name, 'w') as f:
        with redirect_stdout(f):
            model.summary()
    mlflow.log_artifact(temp_name, 'Model Summary')

    try:
        os.remove(temp_name)    
    except FileNotFoundError as e:
        print(f"{temp_name} file is not found")


def previsao(modelo, DirPositivo, DirNegativo):
    ''' Realiza a validacao do modelo
    '''
    ypred = []
    yreal = []
    FP, FN, TP, TN = 0,0,0,0

    for filename in os.listdir(DirPositivo):
        yreal.append(1)
        if "jpg" in filename:
            file_path = os.path.join(DirPositivo, filename)
            img = image.load_img(file_path, target_size=TARGET_SIZE)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x / 255.
            images = np.vstack([x])
            classes = modelo.predict(images)
            #modelo.predict()
            if classes[0][1]>0.5:
                ypred.append(1)
                TP+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')
            else:
                ypred.append(0)
                FP+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')

    for filename in os.listdir(DirNegativo):
        yreal.append(0)
        if "jpg" in filename:
            file_path = os.path.join(DirNegativo, filename)
            img = image.load_img(file_path, target_size=TARGET_SIZE)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x / 255.

            images = np.vstack([x])
            classes = modelo.predict(images)
            if classes[0][0]<=0.5:
                ypred.append(0)
                TN+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')
            else:
                ypred.append(1)
                FN+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')

    log_scalar('TP', TP, 1)
    log_scalar('FP', FP, 1)
    log_scalar('TN', TN, 1)
    log_scalar('FN', FN, 1)

    return yreal, ypred


def calculate_metrics(y_true, y_predict, round_num):
    acuracia = accuracy_score(y_true, y_predict)
    precision = precision_score(y_true, y_predict)
    recall = recall_score(y_true, y_predict)
    f1 = f1_score(y_true, y_predict)

    print("\nAcurácia: {}\nPrecision: {}\nRecall: {}\nF1-Score: {}".format(acuracia, precision, recall, f1))

    log_scalar('accuracy_score', acuracia, round_num)
    log_scalar('precision_score', precision, round_num)
    log_scalar('recall_score', recall, round_num)
    log_scalar('f1_score', f1, round_num)

def log_scalar(name, value, step):
    """Log a scalar value to both MLflow and TensorBoard"""
    mlflow.log_metric(name, value, step=step)

def gerar_matriz_confusao(y_true, y_pred, round_num):
    matriz_conf = matriz_confusao(y_true, y_pred)
    temp_name = "confusion-matrix-(round-" + str(round_num) + ").png"
    matriz_conf.savefig(temp_name)
    mlflow.log_artifact(temp_name, "confusion-matrix-plots")


def matriz_confusao(y_true, y_predict):
    matriz_conf = confusion_matrix(y_true, y_predict)
    fig = plt.figure()
    ax = plt.subplot()
    sns.heatmap(matriz_conf, annot=True, cmap='Blues', ax=ax)

    ax.set_xlabel('Valor Predito')
    ax.set_ylabel('Valor Real')
    ax.set_title('Matriz de Confusão')
    ax.xaxis.set_ticklabels(['0', '1'])
    ax.yaxis.set_ticklabels(['0', '1'])
    plt.close()
    return fig


In [ ]:
class accCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

### Main

In [ ]:
model = load_inceptionv3()
#model = load_mobilenetv2()
#model = load_resnet152v2()
#model = load_vgg16()

In [ ]:
history = model.fit(
    train_dataset(),
    epochs=40,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/40
92/92 [==============================] - 868s 9s/step - loss: 0.4049 - accuracy: 0.8295 - val_loss: 0.0940 - val_accuracy: 0.9856
Epoch 2/40
92/92 [==============================] - 839s 9s/step - loss: 0.1222 - accuracy: 0.9518 - val_loss: 0.0456 - val_accuracy: 0.9896
Epoch 3/40
92/92 [==============================] - 841s 9s/step - loss: 0.0514 - accuracy: 0.9819 - val_loss: 0.0326 - val_accuracy: 0.9904
Epoch 4/40
92/92 [==============================] - 843s 9s/step - loss: 0.0309 - accuracy: 0.9915 - val_loss: 0.0350 - val_accuracy: 0.9920
Epoch 5/40
92/92 [==============================] - 838s 9s/step - loss: 0.0235 - accuracy: 0.9908 - val_loss: 0.0126 - val_accuracy: 0.9984
Epoch 6/40
92/92 [==============================] - ETA: 0s - loss: 0.0120 - accuracy: 0.9962
Reached 99.5% accuracy so cancelling training!
92/92 [==============================] - 840s 9s/step - loss: 0.0120 -

In [ ]:
loss, accuracy, f1score = model.evaluate(test_images, test_labels, verbose=1)

print(accuracy, f1score)

In [ ]:
mlflow.log_param('round_num', 0)
mlflow.log_param('epochs', 40)

In [ ]:
try:
  mlflow.tensorflow.log_model(model)
except:
  print('não deu para salvar o log_modelo')

não deu para salvar o log_modelo


In [ ]:
try:
  save_model(model)
except:
  print('não deu para salvar o modelo')

INFO:tensorflow:Assets written to: /content/sample_data/output/assets


INFO:tensorflow:Assets written to: /content/sample_data/output/assets


não deu para salvar o modelo


In [ ]:
try:
  save_summary(model)
except:
  print('não deu para salvar o summary')

In [ ]:
y_true, y_pred = previsao(model,
                          DATASET_VALID_PATH + VIOLENCIA_DIR,
                          DATASET_VALID_PATH + N_VIOLENCIA_DIR)

TP: 622; FP: 4; TN: 1; FN: 625

In [ ]:
try:
  calculate_metrics(y_true, y_pred, 1)
except:
  print('não deu para calcular as metrias')


Acurácia: 0.4976038338658147
Precision: 0.4987971130713713
Recall: 0.9936102236421726
F1-Score: 0.6641751201281367


In [ ]:
try:
  gerar_matriz_confusao(y_true, y_pred, 1)
except:
  print('não deu para gerar a matriz de confusao')

In [ ]:
log_scalar('std_acc', np.std(history.history['accuracy']), 1)
log_scalar('std_val_acc', np.std(history.history['val_accuracy']), 1)
log_scalar('accuracy', history.history['accuracy'][-1], 1)
log_scalar('val_accuracy', history.history['val_accuracy'][-1], 1)

In [ ]:
mlflow.end_run()

### Generating Friedman

In [ ]:
!pip install orange3

     |████████████████████████████████| 29.0 MB 14.0 MB/s 
     |████████████████████████████████| 992 kB 52.5 MB/s 
     |████████████████████████████████| 228 kB 59.8 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 492 kB 56.6 MB/s 
     |████████████████████████████████| 2.5 MB 46.6 MB/s 
     |████████████████████████████████| 149 kB 72.3 MB/s 
     |████████████████████████████████| 8.3 MB 53.9 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 252 kB 54.9 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
     |████████████████████████████████| 59.9 MB 1.3 MB/s 
     |████████████████████████████████| 338 kB 56.5 MB/s 
     |███████████████████

In [ ]:
from Orange.evaluation import compute_CD, graph_ranks
from scipy.stats import friedmanchisquare, rankdata
import pandas as pd
import numpy as np

In [ ]:
records = [
  {
      'cnn': 'cnn 1',
      'accuracy': 85.66588777, #[85.66588777, 95.66588777, 95.66588777],
      'f1_score': 0.98874541,
  },
  {
      'cnn': 'cnn 2',
      'accuracy': 95.66588777, #[95.66588777, 65.3265487, 78.9456123],
      'f1_score': 0.788874541,
  },
  {
      'cnn': 'cnn 3',
      'accuracy': 90.11154312, #[90.11154, 83.47968574, 61.5847821],
      'f1_score': 0.897568487,
  },
  {
      'cnn': 'cnn 4',
      'accuracy': 80.98765431, #[80.9876543, 91.254789, 92.5874154],
      'f1_score': 0.789654845,
  }
]

In [ ]:
data = pd.DataFrame.from_records(records)
data

,cnn,accuracy,f1_score
0,cnn 1,85.665888,0.988745
1,cnn 2,95.665888,0.788875
2,cnn 3,90.111543,0.897568
3,cnn 4,80.987654,0.789655


In [ ]:
objectives = ['accuracy'] #, 'f1_score'
alpha = 0.05
dataset = 'hear_dataset'
 
print('Null hypothesis:', 'The means of the results of two or more algorithms are the same.')

cnns = data.cnn.unique()
print('\t', 'cnns:', cnns)

Null hypothesis: The means of the results of two or more algorithms are the same.
	 cnns: ['cnn 1' 'cnn 2' 'cnn 3' 'cnn 4']


In [ ]:
for objective in objectives:
  
  print('Dataset, objective:', dataset, objective)
  
  df1 = pd.DataFrame({cnn: list(data[data.cnn == cnn][objective]) for cnn in cnns})
  
  values = df1.values #[90.11154, 83.47968574, 61.5847821]
  names = df1.columns
  print('\t', 'values:', values)
  print('\t', '*values:', *values)
  print('\t', 'names:', list(names))

  friedman = friedmanchisquare(*values)
  ranks = np.array([rankdata(-p) for p in values])
  average_ranks = np.mean(ranks, axis = 0)
  
  cd = compute_CD(average_ranks, n=len(df1), alpha=str(alpha), test='nemenyi')
  
  print('\t', 'null hypothesis:', 'rejected' if friedman.pvalue < alpha else 'accepted')
  print('\t', 'p-value:', friedman.pvalue)
  print('\t', 'ranking:', average_ranks)
  print('\t', 'names:', list(names))
  print('\t', 'cd:', cd)
  
  graph_ranks(average_ranks, names=names, cd=cd)
  # title = f'Friedman-Nemenyi (CD = {round(cd, 3)})'
  title = f'Dataset: {dataset}, Metric: {objective}, CD: {round(cd, 3)}'
  plt.title(title)
  plt.suptitle('p-value: {:.2e}'.format(friedman.pvalue))
  plt.savefig(f'/content/cd_{dataset}_{objective}.pdf', bbox_inches='tight')
  plt.show()


Dataset, objective: hear_dataset accuracy
	 values: [[85.66588777 95.66588777 90.11154312 80.98765431]]
	 *values: [85.66588777 95.66588777 90.11154312 80.98765431]
	 names: ['cnn 1', 'cnn 2', 'cnn 3', 'cnn 4']


ValueError: ignored

In [ ]:
a = np.array([[1, 2, 3], [2, 3, 4] ,[4, 5, 6]])
print('\t', 'a:', a)

friedmanchisquare(*a)

	 a: [[1 2 3]
 [2 3 4]
 [4 5 6]]


FriedmanchisquareResult(statistic=6.0, pvalue=0.04978706836786395)

In [ ]:
performances = pd.read_csv('/content/sample_data/dados.csv')
performances

,cnn,accuracy,Unnamed: 2,f1_score,Unnamed: 4,size,Unnamed: 6,num_layers,Unnamed: 8,num_params,Unnamed: 10,time,Unnamed: 12
0,Assunção et al. (2018),995.150,1.040,995.203,1.026,43.659.935,219.000,18.0,0.0,11427338.0,0.0,1261.45,41.231.663
1,DenseNet169,995.050,1.075,994.951,1.135,146.397.614,21.000,598.0,0.0,12653258.0,0.0,4823.65,65.555.340
2,Diniz et al. (2018),985.725,1.626,985.516,1.745,324.272,0.000,6.0,0.0,25258.0,0.0,1050.35,46.558.707
3,EfficientNetB1,992.975,1.997,992.764,2.136,76.497.769,9.000,343.0,0.0,6587469.0,0.0,3204.80,172.296.745
4,InceptionV3,995.275,980.000,995.146,884.000,250.841.461,2.557,314.0,0.0,21822698.0,0.0,3170.75,144.197.553
5,Lima et al. (2019),988.075,2.092,987.897,2.221,2.013.166,58.000,7.0,0.0,172042.0,0.0,1038.80,41.331.268
6,ResNet50V2,994.675,693.000,994.519,828.000,270.393.280,7.000,193.0,0.0,23579018.0,0.0,2412.80,130.293.838
7,Silva et al. (2021a),993.750,925.000,993.558,1.029,41.098.621,16.000,19.0,0.0,3582506.0,0.0,1194.75,50.831.895
8,Silva et al. (2021b),995.025,850.000,995.035,858.000,30.486.885,24.000,26.0,0.0,2651690.0,0.0,1281.45,36.469.851


In [ ]:
df = pd.DataFrame.from_records(performances)
df

,cnn,accuracy,Unnamed: 2,f1_score,Unnamed: 4,size,Unnamed: 6,num_layers,Unnamed: 8,num_params,Unnamed: 10,time,Unnamed: 12
0,Assunção et al. (2018),995.150,1.040,995.203,1.026,43.659.935,219.000,18.0,0.0,11427338.0,0.0,1261.45,41.231.663
1,DenseNet169,995.050,1.075,994.951,1.135,146.397.614,21.000,598.0,0.0,12653258.0,0.0,4823.65,65.555.340
2,Diniz et al. (2018),985.725,1.626,985.516,1.745,324.272,0.000,6.0,0.0,25258.0,0.0,1050.35,46.558.707
3,EfficientNetB1,992.975,1.997,992.764,2.136,76.497.769,9.000,343.0,0.0,6587469.0,0.0,3204.80,172.296.745
4,InceptionV3,995.275,980.000,995.146,884.000,250.841.461,2.557,314.0,0.0,21822698.0,0.0,3170.75,144.197.553
5,Lima et al. (2019),988.075,2.092,987.897,2.221,2.013.166,58.000,7.0,0.0,172042.0,0.0,1038.80,41.331.268
6,ResNet50V2,994.675,693.000,994.519,828.000,270.393.280,7.000,193.0,0.0,23579018.0,0.0,2412.80,130.293.838
7,Silva et al. (2021a),993.750,925.000,993.558,1.029,41.098.621,16.000,19.0,0.0,3582506.0,0.0,1194.75,50.831.895
8,Silva et al. (2021b),995.025,850.000,995.035,858.000,30.486.885,24.000,26.0,0.0,2651690.0,0.0,1281.45,36.469.851


In [ ]:
objectives = ['accuracy', 'f1_score']
alpha = 0.05
 
print('Null hypothesis:', 'The means of the results of two or more algorithms are the same.')

dataset = '-'
  
data = df
cnns = data.cnn.unique()

for objective in objectives:
  
  print('Dataset, objective:', dataset, objective)
  
  df1 = pd.DataFrame({cnn: list(data[data.cnn == cnn][objective]) for cnn in cnns})
  values = df1.values
  names = df1.columns    
  print('\t', 'values:', values)
  print('\t', '*values:', *values)
  print('\t', 'names:', list(names))

  friedman = friedmanchisquare(*values)
  ranks = np.array([rankdata(-p) for p in values])
  average_ranks = np.mean(ranks, axis = 0)
  
  cd = compute_CD(average_ranks, n=len(df1), alpha=str(alpha), test='nemenyi')
  
  print('\t', 'null hypothesis:', 'rejected' if friedman.pvalue < alpha else 'accepted')
  print('\t', 'p-value:', friedman.pvalue)
  print('\t', 'ranking:', average_ranks)
  print('\t', 'names:', list(names))
  print('\t', 'cd:', cd)
  
  graph_ranks(average_ranks, names=names, cd=cd)
  # title = f'Friedman-Nemenyi (CD = {round(cd, 3)})'
  title = f'Dataset: {dataset}, Metric: {objective}, CD: {round(cd, 3)}'
  plt.title(title)
  plt.suptitle('p-value: {:.2e}'.format(friedman.pvalue))
  plt.savefig(f'/content/cd_{dataset}_{objective}.pdf', bbox_inches='tight')
  plt.show()


Null hypothesis: The means of the results of two or more algorithms are the same.
Dataset, objective: - accuracy
	 values: [[995.15  995.05  985.725 992.975 995.275 988.075 994.675 993.75  995.025]]
	 *values: [995.15  995.05  985.725 992.975 995.275 988.075 994.675 993.75  995.025]
	 names: ['Assunção et al. (2018)', 'DenseNet169', 'Diniz et al. (2018)', 'EfficientNetB1', 'InceptionV3', 'Lima et al. (2019)', 'ResNet50V2', 'Silva et al. (2021a)', 'Silva et al. (2021b)']


ValueError: ignored